In [2]:
library(dplyr)
library(hipathia)
library(survival)
library(survminer)

In [3]:
proj_tcga_sel <- c("tcga_brca", "tcga_coad","tcga_esca","tcga_lgg","tcga_luad","tcga_lusc","tcga_paad","tcga_skcm","tcga_stad")

In [4]:
get_coxmodel_funct <- function(project){
    file <- paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/07_survival_analysis/",project,"/coxph_zph_test_pvadj_sex_",project,".rds",sep="")
    coxmodel <- data.frame(readRDS(file))
    coxmodel$project <- rep(project,length(coxmodel$pvadj_zphtest))
    coxmodel$path <- rownames(coxmodel)
    return(coxmodel)
}

In [5]:
pvadj_coxmodel <- do.call(rbind,lapply(proj_tcga_sel, FUN=function(x) get_coxmodel_funct(x)))

In [6]:
pvadj_coxmodel_sign <- pvadj_coxmodel %>% filter(pvadj_coxph < 0.05 & pvadj_zphtest < 0.05)

In [7]:
limma_pathvals <- readRDS("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/03_limma_pathvals/limma_pathvals.rds")
limma_pathvals_sign <- limma_pathvals %>% filter(adj.P.Val < 0.05)

In [8]:
pvadj_coxmodel_sign$sex_limma <- unlist(lapply(1:length(pvadj_coxmodel_sign[,"project"]), FUN=function(x)
paste(data.frame(limma_pathvals_sign %>% filter(project == pvadj_coxmodel_sign[x,"project"] & 
                                                circ==pvadj_coxmodel_sign[x,"path"]))$sex, collapse=", ")))

In [9]:
pvadj_coxmodel_sign <- pvadj_coxmodel_sign %>% mutate(project_name = case_when(project == "tcga_brca" ~ "Breast invasive carcinoma",
                                    project == "tcga_coad" ~ "Colon adenocarcinoma",
                                    project == "tcga_esca" ~ "Esophageal carcinoma",
                                    project == "tcga_lgg" ~ "Brain Lower Grade Glioma",
                                    project == "tcga_luad" ~ "Lung adenocarcinoma",
                                    project == "tcga_lusc" ~ "Lung squamous cell carcinoma",
                                    project == "tcga_paad" ~ "Pancreatic Adenocarcinoma",
                                    project == "tcga_skcm" ~ "Skin Cutaneous Melanoma",
                                    project == "tcga_stad" ~ "Stomach adenocarcinoma"))

In [10]:
pvadj_coxmodel_sign_save <- pvadj_coxmodel_sign[,c("project_name","path","sex_limma")]

In [11]:
paths_hallmarks <- read.table("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/data/paths_hallmarks.tsv",sep="\t",header = T)
rownames(paths_hallmarks) <- paths_hallmarks$X.1

In [12]:
pvadj_coxmodel_sign_save$hallmark <- unlist(lapply(pvadj_coxmodel_sign_save$path, FUN=function(x)
paste(gsub("."," ", tolower(colnames(paths_hallmarks[-c(1,2)])[abs(na.omit(paths_hallmarks[x,-c(1,2)])) == 1]),fixed=T), collapse=", ")))

In [13]:
write.csv(pvadj_coxmodel_sign_save, file="/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/08_analysis_coxmodels/coxmodel_sign.csv", row.names=F)  

In [14]:
rownames(pvadj_coxmodel_sign_save) <- NULL
pvadj_coxmodel_sign_save

project_name,path,sex_limma,hallmark
<chr>,<chr>,<chr>,<chr>
Breast invasive carcinoma,Vasopressin-regulated water reabsorption: ARHGDIA,"male, female","immune destruction, cellular energetics, replicative immortality, evading growth suppressors, inducing angiogenesis, sustaining proliferative signaling, tumor promoting inflammation"
Colon adenocarcinoma,Adipocytokine signaling pathway: PTPN11,"male, female",cellular energetics
Colon adenocarcinoma,Adipocytokine signaling pathway: POMC,"male, female",
Brain Lower Grade Glioma,Glucagon signaling pathway: PYGB,male,"invasion and metastasis, cellular energetics, replicative immortality, evading growth suppressors, inducing angiogenesis, sustaining proliferative signaling"
Brain Lower Grade Glioma,Glucagon signaling pathway: PFKFB1,,"invasion and metastasis, immune destruction, replicative immortality, evading growth suppressors, inducing angiogenesis, resisting cell death, sustaining proliferative signaling, tumor promoting inflammation"
Lung adenocarcinoma,Sphingolipid signaling pathway: SMPD1,"male, female","invasion and metastasis, immune destruction, cellular energetics, replicative immortality, evading growth suppressors, tumor promoting inflammation"
Pancreatic Adenocarcinoma,Oxytocin signaling pathway: Arachidonate,"male, female",
Skin Cutaneous Melanoma,cAMP signaling pathway: Phosphatidate,"male, female",
Skin Cutaneous Melanoma,Gap junction: GJA1 GJA1**,"male, female","invasion and metastasis, immune destruction, cellular energetics, replicative immortality, evading growth suppressors, genome instability and mutation, inducing angiogenesis, resisting cell death, sustaining proliferative signaling, tumor promoting inflammation"


In [15]:
### survival plot

In [18]:
print_survplot_funct <- function(project) {
    
    # Load necessary libraries if not already loaded
    library(survival)
    library(survminer)
    library(ggsci)  # Load ggsci for AAAS palette
    
    pathways <- load_pathways("hsa")
    
    pats_geneexp_clind_drug_samptype_file <- paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/02_selection_samples/02_1_ext_information_patients_tcga/", project, "/patients_geneexp_clind_drug_samptype_", project, ".rds", sep = "")
    
    pats_geneexp_clind_drug_samptype <- readRDS(pats_geneexp_clind_drug_samptype_file)
    
    pats_geneexp_clind_drug_samptype$time <- pats_geneexp_clind_drug_samptype$days_to_last_followup
    pats_geneexp_clind_drug_samptype$time[is.na(pats_geneexp_clind_drug_samptype$days_to_last_followup)] <- pats_geneexp_clind_drug_samptype$days_to_death[is.na(pats_geneexp_clind_drug_samptype$days_to_last_followup)]
    
    patient_sex_rec_time <- unique(pats_geneexp_clind_drug_samptype %>% 
                                    filter(sample_type == "Primary Solid Tumor") %>% 
                                    select(gender, has_new_tumor_events_information, patient, time))
    
    patient_sex_rec_time <- patient_sex_rec_time %>% 
      mutate(bi_has_new_tumor_events_information = case_when(
        has_new_tumor_events_information == "YES" ~ 1,
        has_new_tumor_events_information == "NO" ~ 0
      ))
    
    # Read pathvals from file
    pathvals_file <- paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/01_mechanistics_models/01_1_mechanistics_models_tcga/", project, "/pathvals_", project, ".rds", sep = "")
    pathvals <- readRDS(pathvals_file)
    
    # Read physiological_paths
    physiological_paths <- read.table("/mnt/lustre/scratch/CBRA/projects/heterogeneity/data/physiological_paths.tsv", sep = "\t")
    
    # Filter pathvals for tumors
    pathvals_tumor <- pathvals[unlist(lapply(physiological_paths$V2, FUN = function(x) grep(x, rownames(pathvals)))), as.data.frame(pats_geneexp_clind_drug_samptype %>% filter(sample_type == "Primary Solid Tumor"))$patient]
    
    # Scale values of pathvals selecting +-0.5
    pathvals_tumor_sel_scale <- t(pathvals_tumor[rownames(pathvals_tumor)[apply(abs(scale(pathvals_tumor)) > 0.5, 1, sum) == length(colnames(pathvals_tumor))], ])
    colnames(pathvals_tumor_sel_scale) <- get_path_names(pathways, colnames(pathvals_tumor_sel_scale))
    
    pat_id <- rownames(pathvals_tumor_sel_scale)
    pathvals_tumor_sel_scale <- as.data.frame(pathvals_tumor_sel_scale)
    pathvals_tumor_sel_scale$patients <- pat_id
    
    # Merge dataframes for calculations
    patient_sex_rec_time_circ_scale <- merge(patient_sex_rec_time, pathvals_tumor_sel_scale, by.x = "patient", by.y = "patients")
    pathvals_tumor_sel_scale$patients <- NULL
    
    circs_sign <- pvadj_coxmodel_sign[pvadj_coxmodel_sign$project == project, "path"]
    
    # Function to compute the p-value of the interaction term in Cox PH model with SEX
    pvalue_coxph_circ_sex <- function(x, project) {
      
      # Construct the formula as a string and then convert it to a formula object
      formula_str <- paste0("Surv(time, bi_has_new_tumor_events_information) ~ gender * `", x, "`")
      coxph_model <- coxph(as.formula(formula_str), data = patient_sex_rec_time_circ_scale)
      
      # Predict survival curves based on the new data
      newdata <- expand.grid(
        gender = levels(patient_sex_rec_time_circ_scale$gender),
        circuits = quantile(patient_sex_rec_time_circ_scale[[x]], probs = c(0.25, 0.75))
      )
      colnames(newdata)[2] <- x  # Ensure the correct column name is used
      surv_fit <- survfit(coxph_model, newdata = newdata)
      
      # Plot survival curves with AAAS palette
      surplot <- ggsurvplot(
        surv_fit,
        data = patient_sex_rec_time_circ_scale,
        pval = TRUE,  # Show p-value on the plot
        conf.int = TRUE,  # Show confidence intervals
        legend.labs = paste(newdata$gender, round(newdata[[x]], 2)),  # Label legend with gender and circuits values
        xlab = "Time",
        ylab = "Survival Probability",
        palette = "aaas"  # Use AAAS palette
      )
      
      # Save the plot as PNG
      filename <- paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/08_analysis_coxmodels/survival_plot_", project, "_", x, ".png", sep = "")
      jpeg(filename, quality = 100, width = 850, height = 700)
        print(surplot)
      dev.off()
      # Return a message indicating successful saving
      message(paste("Plot saved as:", filename))
    }
    
    # Generate survival plots for each circuit in circs_sign
    invisible(lapply(circs_sign, FUN = function(x) pvalue_coxph_circ_sex(x, project)))
}


In [19]:
lapply(proj_tcga_sel, FUN=function(x) print_survplot_funct(x))

Loaded 146 pathways

Warning message in .pvalue(fit, data = data, method = method, pval = pval, pval.coord = pval.coord, :
“There are no survival curves to be compared. 
 This is a null model.”
Plot saved as: /mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/08_analysis_coxmodels/survival_plot_tcga_brca_Vasopressin-regulated water reabsorption: ARHGDIA.png

Loaded 146 pathways

Warning message in .pvalue(fit, data = data, method = method, pval = pval, pval.coord = pval.coord, :
“There are no survival curves to be compared. 
 This is a null model.”
Plot saved as: /mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/08_analysis_coxmodels/survival_plot_tcga_coad_Adipocytokine signaling pathway: PTPN11.png

Warning message in .pvalue(fit, data = data, method = method, pval = pval, pval.coord = pval.coord, :
“There are no survival curves to be compared. 
 This is a null model.”
Plot saved as: /mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/08_analysis_coxmodels/sur

[[1]]
[[1]][[1]]
NULL


[[2]]
[[2]][[1]]
NULL

[[2]][[2]]
NULL


[[3]]
list()

[[4]]
[[4]][[1]]
NULL

[[4]][[2]]
NULL


[[5]]
[[5]][[1]]
NULL


[[6]]
list()

[[7]]
[[7]][[1]]
NULL


[[8]]
[[8]][[1]]
NULL

[[8]][[2]]
NULL

[[8]][[3]]
NULL

[[8]][[4]]
NULL

[[8]][[5]]
NULL

[[8]][[6]]
NULL

[[8]][[7]]
NULL


[[9]]
list()